In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,LSTM,Dropout,BatchNormalization,Embedding,TimeDistributed,Activation
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from sklearn.model_selection import train_test_split
from tensorflow.keras import regularizers
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau




In [2]:
sentences3 = []

with open("/kaggle/input/subtitles-dataset/en (1).txt", "r", encoding="utf-8") as f:
    for i, line in enumerate(f):
        if i >= 500000:   
            break
        line = line.strip()
        if line:
            sentences3.append(line.lower())


train_sentences, val_sentences = train_test_split(sentences3, test_size=0.2, random_state=42)
print("Train sentences:", len(train_sentences))
print("Validation sentences:", len(val_sentences))

Train sentences: 400000
Validation sentences: 100000


In [3]:
tokenizer3 = Tokenizer(oov_token="<OOV>")
tokenizer3.fit_on_texts(train_sentences)  
total_words_3 = len(tokenizer3.word_index) + 1
print("Total vocab size:", total_words_3)


def build_ngram_sequences(lines, tokenizer, max_len=None):
    sequences = []
    for line in lines:
        token_list = tokenizer.texts_to_sequences([line])[0]
        for i in range(1, len(token_list)):
            n_gram = token_list[:i+1]
            sequences.append(n_gram)
    if not sequences:
        return None, None, max_len 

    if max_len is None:
        max_len = max(len(x) for x in sequences)

 
    sequences = pad_sequences(sequences, maxlen=max_len, padding="pre")
    X, y = sequences[:, :-1], sequences[:, -1]
    return X, y, max_len


X3, y3, max_len_seq3 = build_ngram_sequences(train_sentences, tokenizer3)
print("Train samples:", X3.shape, y3.shape)
print("Max sequence length:", max_len_seq3)

X_val, y_val, _ = build_ngram_sequences(val_sentences, tokenizer3, max_len=max_len_seq3)
print("Validation samples:", X_val.shape, y_val.shape)


Total vocab size: 31572
Train samples: (1773893, 59) (1773893,)
Max sequence length: 60
Validation samples: (444740, 59) (444740,)


In [5]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, GRU, Dense, BatchNormalization, Dropout
from tensorflow.keras import regularizers

model = Sequential()
model.add(Embedding(total_words_3, 128, input_length=max_len_seq3-1))
model.add(GRU(128, return_sequences=True, dropout=0.3))
model.add(GRU(128, dropout=0.3))

model.add(Dense(256, activation="relu", kernel_regularizer=regularizers.l2(0.001)))
model.add(BatchNormalization())
model.add(Dense(128, activation="relu", kernel_regularizer=regularizers.l2(0.001)))
model.add(BatchNormalization())
model.add(Dropout(0.4))


model.add(Dense(total_words_3, activation="softmax"))

model.compile(
    loss="sparse_categorical_crossentropy",
    optimizer='adam',
    metrics=["accuracy"]
)

In [6]:
model.summary()

<Sequential name=sequential_1, built=False>

In [ ]:


# 🔹 Callbacks
early_stop = EarlyStopping(monitor="val_loss", patience=3, restore_best_weights=True)
lr_scheduler = ReduceLROnPlateau(monitor="val_loss", factor=0.5, patience=2, verbose=1)

# Training
history = model.fit(
    X3, y3,
    epochs=30,
    batch_size=64,    
    validation_data=(X_val, y_val),
    callbacks=[early_stop, lr_scheduler]
)


In [10]:
cnn.save("gru_mode.h5")

In [7]:
import pickle
with open("tokenizer.pkl", "wb") as f:
    pickle.dump(tokenizer3, f)



In [ ]:


import pickle


with open("tokenizer.pkl", "rb") as f:
    tok3 = pickle.load(f)


In [10]:
from tensorflow.keras.models import load_model

# path to your model file (use quotes and correct directory)
model_path = "/kaggle/input/gru-model/tensorflow2/default/1/gru_mode.h5"

# load the model
model1 = load_model(model_path)

# verify load
model1.summary()


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ (None, 59, 128)        │     4,041,216 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ gru (GRU)                       │ (None, 59, 128)        │        99,072 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ gru_1 (GRU)                     │ (None, 128)            │        99,072 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 256)            │        33,024 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 256)            │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 128)            │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 128)            │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 31572)          │     4,072,788 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 8,379,606 (31.97 MB)

 Trainable params: 8,378,836 (31.96 MB)

 Non-trainable params: 768 (3.00 KB)

 Optimizer params: 2 (12.00 B)

In [11]:
def output(seed_text, model, max_len, tokenizer, next_words=20):
    in_text = seed_text

    for _ in range(next_words):
        seq = tokenizer.texts_to_sequences([in_text])[0]  # fixed typo
        seq = seq[-(max_len-1):]  # keep last max_len-1 tokens
        seq = np.pad(seq, (max_len-1-len(seq), 0), "constant")

        y_pred = model.predict(seq.reshape(1, max_len-1), verbose=0)  # fixed typo
        next_index = np.argmax(y_pred)
        next_word = tokenizer.index_word[next_index]

        in_text += " " + next_word

    return in_text


In [24]:
print(output(
    seed_text="elephant",
    next_words=3,
    model=model1,
    max_len=max_len_seq3,
    tokenizer=tokenizer3,
   
))

elephant is the one
